# 1-D Framework
<p>This notebook represents the cumulative and current state of the 1D functions that I have yet created.</p>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
-
class FlowModel:
    
    def __init__(self, length = 10, numElements = 10, self.conductivity = .1):
        self.length = length
        self.numElements = numElements
        self.conductivity = conductivity
        self.heads = np.zeros(numElements)
        self.scale = length/numElements
        
    def flow(self):
        '''
        Normal 1D flow equation
        '''
        queue = np.zeros(self.numElements)
        queue[:-1] += conductivity*(self.heads[1: ] - self.heads[:-1])/self.scale
        queue[1: ] += conductivity*(self.heads[:-1] - self.heads[1: ])/self.scale
        heads += queue
        del queue